In [ ]:
!nvidia-smi

Wed Aug  3 17:55:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from collections import OrderedDict
from torch import Tensor
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
import glob
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Subset,DataLoader
import torchvision.transforms as transforms
import torchvision
import random
from google.colab import files
from sklearn.metrics import  confusion_matrix
import sklearn
from sklearn.model_selection import ShuffleSplit
import cv2
from google.colab.patches import cv2_imshow
from scipy.ndimage import distance_transform_edt
from torch.autograd import Variable
import skimage.segmentation
import skimage.io
import skimage 
from scipy.optimize import linear_sum_assignment
import skimage.segmentation
import matplotlib.pyplot as plt
import skimage.io
import skimage.segmentation
from skimage import feature
from skimage import filters
import copy
import torchvision
from collections import OrderedDict
import math
import imageio

In [2]:
path='/content/drive/MyDrive/Histopathology_Datasets/Nuclei_Datasets/Nuclei_Dataset1_Segmentation_Regression'

In [3]:
l=os.listdir('/content/drive/MyDrive/Histopathology_Datasets/Nuclei_Datasets/Nuclei_Dataset1_Segmentation_Regression')
l.sort()

In [4]:
train_image=[]
train_label=[]
test_image=[]
test_label=[]
for i in range(11):
  image_path=path+'/'+l[i+12]
  image_path=glob.glob(image_path+'*/**.png')
  image_path.sort()
  train_image_path,test_image_path=sklearn.model_selection.train_test_split(image_path,test_size=0.2,random_state=1154)
  train_image.extend(train_image_path)
  test_image.extend(test_image_path)

  label_path=path+'/'+l[i+1]
  label_path=glob.glob(label_path+'*/**.png')
  label_path.sort()
  train_label_path,test_label_path=sklearn.model_selection.train_test_split(label_path,test_size=0.2,random_state=1154)
  train_label.extend(train_label_path)
  test_label.extend(test_label_path)

In [9]:
def extract_patches_2d(img,patch_shape,step=[1.0,1.0],batch_first=False):
    patch_H, patch_W = patch_shape[0], patch_shape[1]
    if(img.size(2)<patch_H):
        num_padded_H_Top = (patch_H - img.size(2))//2
        num_padded_H_Bottom = patch_H - img.size(2) - num_padded_H_Top
        padding_H = nn.ConstantPad2d((0,0,num_padded_H_Top,num_padded_H_Bottom),0)
        img = padding_H(img)
    if(img.size(3)<patch_W):
        num_padded_W_Left = (patch_W - img.size(3))//2
        num_padded_W_Right = patch_W - img.size(3) - num_padded_W_Left
        padding_W = nn.ConstantPad2d((num_padded_W_Left,num_padded_W_Right,0,0),0)
        img = padding_W(img)
    step_int = [0,0]
    step_int[0] = int(patch_H*step[0]) if(isinstance(step[0], float)) else step[0]
    step_int[1] = int(patch_W*step[1]) if(isinstance(step[1], float)) else step[1]
    patches_fold_H = img.unfold(2, patch_H, step_int[0])
    if((img.size(2) - patch_H) % step_int[0] != 0):
        patches_fold_H = torch.cat((patches_fold_H,img[:,:,-patch_H:,].permute(0,1,3,2).unsqueeze(2)),dim=2)
    patches_fold_HW = patches_fold_H.unfold(3, patch_W, step_int[1])   
    if((img.size(3) - patch_W) % step_int[1] != 0):
        patches_fold_HW = torch.cat((patches_fold_HW,patches_fold_H[:,:,:,-patch_W:,:].permute(0,1,2,4,3).unsqueeze(3)),dim=3)
    patches = patches_fold_HW.permute(2,3,0,1,4,5)
    patches = patches.reshape(-1,img.size(0),img.size(1),patch_H,patch_W)
    if(batch_first):
        patches = patches.permute(1,0,2,3,4)
    return patches

def reconstruct_from_patches_2d(patches,img_shape,step=[1.0,1.0],batch_first=False):
    if(batch_first):
        patches = patches.permute(1,0,2,3,4)
    patch_H, patch_W = patches.size(3), patches.size(4)
    img_size = (patches.size(1), patches.size(2),max(img_shape[0], patch_H), max(img_shape[1], patch_W))
    step_int = [0,0]
    step_int[0] = int(patch_H*step[0]) if(isinstance(step[0], float)) else step[0]
    step_int[1] = int(patch_W*step[1]) if(isinstance(step[1], float)) else step[1]
    nrow, ncol = 1 + (img_size[-2] - patch_H)//step_int[0], 1 + (img_size[-1] - patch_W)//step_int[1]
    r_nrow = nrow + 1 if((img_size[2] - patch_H) % step_int[0] != 0) else nrow
    r_ncol = ncol + 1 if((img_size[3] - patch_W) % step_int[1] != 0) else ncol
    patches = patches.reshape(r_nrow,r_ncol,img_size[0],img_size[1],patch_H,patch_W)
    img = torch.zeros(img_size, device = patches.device)
    overlap_counter = torch.zeros(img_size, device = patches.device)
    for i in range(nrow):
        for j in range(ncol):
            img[:,:,i*step_int[0]:i*step_int[0]+patch_H,j*step_int[1]:j*step_int[1]+patch_W] += patches[i,j,]
            overlap_counter[:,:,i*step_int[0]:i*step_int[0]+patch_H,j*step_int[1]:j*step_int[1]+patch_W] += 1
    if((img_size[2] - patch_H) % step_int[0] != 0):
        for j in range(ncol):
            img[:,:,-patch_H:,j*step_int[1]:j*step_int[1]+patch_W] += patches[-1,j,]
            overlap_counter[:,:,-patch_H:,j*step_int[1]:j*step_int[1]+patch_W] += 1
    if((img_size[3] - patch_W) % step_int[1] != 0):
        for i in range(nrow):
            img[:,:,i*step_int[0]:i*step_int[0]+patch_H,-patch_W:] += patches[i,-1,]
            overlap_counter[:,:,i*step_int[0]:i*step_int[0]+patch_H,-patch_W:] += 1
    if((img_size[2] - patch_H) % step_int[0] != 0 and (img_size[3] - patch_W) % step_int[1] != 0):
        img[:,:,-patch_H:,-patch_W:] += patches[-1,-1,]
        overlap_counter[:,:,-patch_H:,-patch_W:] += 1
    img /= overlap_counter
    if(img_shape[0]<patch_H):
        num_padded_H_Top = (patch_H - img_shape[0])//2
        num_padded_H_Bottom = patch_H - img_shape[0] - num_padded_H_Top
        img = img[:,:,num_padded_H_Top:-num_padded_H_Bottom,]
    if(img_shape[1]<patch_W):
        num_padded_W_Left = (patch_W - img_shape[1])//2
        num_padded_W_Right = patch_W - img_shape[1] - num_padded_W_Left
        img = img[:,:,:,num_padded_W_Left:-num_padded_W_Right]
    return img


In [10]:
def image_generator(image_path,labels_path,im_path,label_path):
  for i in range(len(image_path)):
    image=imageio.imread(image_path[i])
    label=imageio.imread(labels_path[i])
    image=torch.from_numpy(image[:,:,0:3])
    label=torch.from_numpy(label)
    image=image.permute(2,0,1).unsqueeze(0)
    label=label.unsqueeze(0).unsqueeze(0)
    images=extract_patches_2d(image,(128,128),step=[1.0,1.0],batch_first=True)
    labels=extract_patches_2d(label,(128,128),step=[1.0,1.0],batch_first=True)
    for j in range(images.shape[1]):
      a=images[0,j,:,:,:].permute(1,2,0)
      b=labels[0,j,:,:,:].permute(1,2,0)
      a=np.array(a).astype('uint8')
      b=np.array(b).astype('uint8')
      img_pth=str(i) + '_' + str(j) + '_' + 'image.png'
      lbl_pth=str(i) + '_' + str(j) + '_' + 'label.png'
      final_image_path=os.path.join(im_path,img_pth)
      final_label_path=os.path.join(label_path,lbl_pth)
      cv2.imwrite(final_image_path,a)
      cv2.imwrite(final_label_path,b)

In [11]:
%mkdir Train_images
%mkdir Train_labels

%mkdir Test_images
%mkdir Test_labels

image_generator(train_image,train_label,'/content/Train_images', '/content/Train_labels')
image_generator(test_image,test_label,'/content/Test_images', '/content/Test_labels')

In [ ]:
image=imageio.imread('/content/Test_labels/12_0_label.png')

In [12]:
def normalizeStaining(img, Io=240, alpha=1, beta=0.15):
    ''' Normalize staining appearence of H&E stained images
    
    Example use:
        see test.py
        
    Input:
        I: RGB input image
        Io: (optional) transmitted light intensity
        
    Output:
        Inorm: normalized image
        H: hematoxylin image
        E: eosin image
    
    Reference: 
        A method for normalizing histology slides for quantitative analysis. M.
        Macenko et al., ISBI 2009
    '''
             
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
        
    maxCRef = np.array([1.9705, 1.0308])
    
    # define height and width of image
    h, w, c = img.shape
    
    # reshape image
    img = img.reshape((-1,3))

    # calculate optical density
    OD = -np.log((img.astype(np.float32)+1)/Io)
    
    # remove transparent pixels
    ODhat = OD[~np.any(OD<beta, axis=1)]
        
    # compute eigenvectors
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
    
    #eigvecs *= -1
    
    #project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3])
    
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    # recreate the image using reference mixing matrix
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
    
    # unmix hematoxylin and eosin
    H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
    H[H>255] = 254
    H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)
    
    E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
    E[E>255] = 254
    E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)

    return Inorm, H, E

In [13]:
class TNBC_train(Dataset):
  def __init__(self):
    self.image_path=glob.glob('/content/Train_images*/**.png')
    self.image_path.sort()
    self.labels=glob.glob('/content/Train_labels*/**.png')
    self.labels.sort()

  def __len__(self):
    return len(self.labels)    

  def __getitem__(self,idx):
    image=imageio.imread(self.image_path[idx])
    if idx not in [81,85,86,90,121,250,410,414,419,420,430,431]:
      image,H,E=normalizeStaining(image)  
    label=imageio.imread(self.labels[idx])
    bound=skimage.segmentation.find_boundaries(label, mode='inner')
    bound=bound*1.0
    label=(label>0)*1.0

    image=torch.from_numpy(image/255.0).permute(2,1,0) 
    label=torch.from_numpy(label).unsqueeze(0)
    bound=torch.from_numpy(bound).unsqueeze(0)
    label=torch.rot90(label, 1, [1,2])
    label=torch.flip(label,[0,1])
    bound=torch.rot90(bound, 1, [1,2])
    bound=torch.flip(bound,[0,1])    
    return image, torch.cat((label,bound),0)    

In [14]:
class TNBC_test(Dataset):
  def __init__(self):
    self.image_path=glob.glob('/content/Test_images*/**.png')
    self.image_path.sort()
    self.labels=glob.glob('/content/Test_labels*/**.png')
    self.labels.sort()

  def __len__(self):
    return len(self.labels)    

  def __getitem__(self,idx):
    image=imageio.imread(self.image_path[idx])
    if idx not in [46]:
      image,H,E=normalizeStaining(image)  
    label=imageio.imread(self.labels[idx])
    bound=skimage.segmentation.find_boundaries(label, mode='inner')
    bound=bound*1.0    
    label=(label>0)*1.0

    image=torch.from_numpy(image/255.0).permute(2,1,0) 
    label=torch.from_numpy(label).unsqueeze(0)
    bound=torch.from_numpy(bound).unsqueeze(0)
    label=torch.rot90(label, 1, [1,2])
    label=torch.flip(label,[0,1])
    bound=torch.rot90(bound, 1, [1,2])
    bound=torch.flip(bound,[0,1])    
    return image, torch.cat((label,bound),0)   

In [15]:
train_ds=TNBC_train()
test_ds=TNBC_test()
train_dl=DataLoader(train_ds,batch_size=16,num_workers=2)
test_dl=DataLoader(test_ds,batch_size=16,num_workers=2)

In [ ]:
class Upsample(nn.Module):
    """ nn.Upsample is deprecated """
 
    def __init__(self, scale_factor, mode="bilinear"):
        super(Upsample, self).__init__()
        self.scale_factor = scale_factor
        self.mode = mode
 
    def forward(self, x):
        x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode, align_corners=True, recompute_scale_factor=True)
        return x    
        
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)
 
 
def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
 

In [ ]:
class bn_conv(nn.Module):
    def __init__(self,num_init_features,output_channels,dilation=1):
        super(bn_conv,self).__init__()
        self.features = nn.Sequential(OrderedDict([ 
            ('conv0', nn.Conv2d(num_init_features,output_channels,kernel_size=3, stride=1,
                                padding=dilation, bias=False,dilation=dilation)), 
            ('norm0', nn.BatchNorm2d(output_channels)),
            ('relu0', nn.ReLU(inplace=True))          
    
        ]))
    def forward(self,x):
        x=self.features(x)
        return x

In [ ]:
class bn_conv2(nn.Module):
    def __init__(self, initial_channels,output_channels,dilation=1):
        super(bn_conv2,self).__init__()
        self.conv1=bn_conv(initial_channels,output_channels,dilation)
        self.conv2=bn_conv(output_channels,output_channels,dilation)
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        return x 

In [ ]:
class Spatial_Attn(nn.Module):
  def __init__(self):
    super(Spatial_Attn,self).__init__()
  
  def forward(self,x):
    x1=torch.mean(x,1,keepdim=True)
    x2,_=torch.max(x,1,keepdim=True)  
    x_out = x*torch.sigmoid(x1) + x*torch.sigmoid(x2)
    return x_out

In [ ]:
class Channel_Attn(nn.Module):
  def __init__(self):
    super(Channel_Attn,self).__init__()
    self.gap=nn.AdaptiveAvgPool2d((1,1))
    self.gmp=nn.AdaptiveMaxPool2d((1,1))
  
  def forward(self,x):
    x1=self.gap(x) 
    x2=self.gmp(x)
    x_out = x*torch.sigmoid(x1) + x*torch.sigmoid(x2)
    return x_out  

In [ ]:
class Attention(nn.Module):
  def __init__(self,input_features):
    super(Attention,self).__init__()
    self.sa=Spatial_Attn()
    self.ca=Channel_Attn()
    self.conv=conv3x3(2*input_features,input_features)
    self.bn=nn.BatchNorm2d(input_features)
  
  def forward(self,x):
    x1=self.sa(x)
    x2=self.ca(x)
    x_attn=torch.cat((x1,x2),1)
    x_attn=self.bn(self.conv(x_attn))
    x_attn=x+x_attn
    x_attn=x*torch.sigmoid(x_attn)
    return x_attn    

In [ ]:
class enc_sup(nn.Module):
  def __init__(self,input_features,upsample_factor=1):
    super(enc_sup,self).__init__()
    self.conv1=nn.Conv2d(input_features,64,kernel_size=1)
    self.upsample=Upsample(upsample_factor)
    self.conv2=nn.Conv2d(64,64,kernel_size=1)
    self.out=nn.Conv2d(64,2,kernel_size=1)
  
  def forward(self,x):
    x=self.conv1(x)
    x=self.upsample(x)
    x=self.conv2(x)
    x_out=self.out(x)
    return torch.sigmoid(x_out)


In [ ]:
class dec_sup(nn.Module):
  def __init__(self,input_features):
    super(dec_sup,self).__init__()
    self.conv1=bn_conv(input_features,64)
    self.conv2=nn.Conv2d(64,64,kernel_size=1)
    self.out=nn.Conv2d(64,2,kernel_size=1)
  
  def forward(self,x):
    x=self.conv1(x)
    x=self.conv2(x)
    x_out=self.out(x)
    return torch.sigmoid(x_out)    

In [ ]:
class BasicBlock(nn.Module):
    expansion: int = 1
 
    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample = True,
        dilation: list = [],
        upsample_factor: int = 1        
    ) -> None:
        super(BasicBlock, self).__init__()
        norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(inplanes, planes, stride=1 ,dilation=dilation[0])
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes,stride=1,dilation=dilation[1])
        self.bn2 = norm_layer(planes)
        # self.conv3 = conv3x3(planes, planes,stride=1,dilation=dilation[2])
        # self.bn3 = norm_layer(planes)        

        self.attn = Attention(planes)
        self.map = enc_sup(planes,upsample_factor)
        if downsample == True:
            self.downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )
        elif isinstance(downsample, nn.Module):
            self.downsample = downsample
        else:
            self.downsample = None
        self.stride = stride
 
    def forward(self, x: Tensor) -> Tensor:
        identity = x
        if self.downsample is not None:
            identity = self.downsample(identity)

        x1 = self.conv1(x)
        x1 = self.bn1(x1)
 
        x2 = self.conv2(x1+identity)
        x2 = self.bn2(x2)

        # x3 = self.conv3(x2)
        # x3 = self.bn3(x3)
 
        # if self.downsample is not None:
        #     identity = self.downsample(identity)
 
        out = x1 + x2 + identity
        out = self.relu(out)
        out = self.attn(out)
        out_map = self.map(out)
        return out, out_map

In [ ]:
class ResNet(nn.Module):
 
    def __init__(
        self,
        initial_channel: int, 
        block: Type[Union[BasicBlock]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
 
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:

            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(initial_channel, self.inplanes, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
 
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1, dilate=False,
                                       dilation=[1,2], upsample_factor=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=1,
                                       dilate=False, dilation=[3,4], upsample_factor=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1,
                                       dilate=False, dilation=[5,6], upsample_factor=4)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1,
                                       dilate=False, dilation=[7,8], upsample_factor=8)
 
        self.maxpool1=nn.MaxPool2d(kernel_size=2,stride=2)
        self.maxpool2=nn.MaxPool2d(kernel_size=2,stride=2)
        self.maxpool3=nn.MaxPool2d(kernel_size=2,stride=2)
        self.maxpool4=nn.MaxPool2d(kernel_size=2,stride=2)
 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
 
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]
 
    def _make_layer(self, block: Type[Union[BasicBlock]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False, dilation: list = [], upsample_factor:int=1,) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        upsample_factor=upsample_factor
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample,
                            dilation=dilation,upsample_factor=upsample_factor))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,stride=1,
                                dilation=dilation,upsample_factor=upsample_factor))
 
        return nn.Sequential(*layers)
 
    def forward(self, x): 
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x1,x1_out = self.layer1(x)
        x2,x2_out = self.layer2(self.maxpool1(x1))
        x3,x3_out = self.layer3(self.maxpool2(x2))
        x4,x4_out = self.layer4(self.maxpool3(x3))
        x5 = self.maxpool4(x4)
 
        return x1,x2,x3,x4,x5,x1_out,x2_out,x3_out,x4_out

In [ ]:
class attn_pyram(nn.Module):
  def __init__(self,input_features):
    super(attn_pyram,self).__init__()
    self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=2)
    self.pool2 = nn.MaxPool2d(kernel_size=[3, 3], stride=3)
    self.pool3 = nn.MaxPool2d(kernel_size=[4, 4], stride=4)
    self.pool4 = nn.MaxPool2d(kernel_size=[5, 5], stride=5)
    self.conv = bn_conv2(4*input_features,input_features,6)
  def forward(self,x):
    in_channels, h, w = x.size(1), x.size(2), x.size(3)
    
    x1 = F.interpolate(self.pool1(x), size=(h, w), mode='bilinear', align_corners=True)
    x1 = x*torch.sigmoid(x1)
    x2 = F.interpolate(self.pool2(x), size=(h, w), mode='bilinear', align_corners=True)
    x2 = x*torch.sigmoid(x2)
    x3 = F.interpolate(self.pool3(x), size=(h, w), mode='bilinear', align_corners=True)
    x3 = x*torch.sigmoid(x3)
    x4 = F.interpolate(self.pool4(x), size=(h, w), mode='bilinear', align_corners=True)    
    x4 = x*torch.sigmoid(x4)

    x_out=torch.cat((x1,x2,x3,x4),1)
    x_out=self.conv(x_out)
    return x_out


In [ ]:
class upconv_block(nn.Module):
  def __init__(self,input_features,output_features):
    super(upconv_block,self).__init__()
    self.conv1=nn.ConvTranspose2d(input_features,output_features,kernel_size=2,stride=2)
    self.attn=Attention(output_features)
  def forward(self,x):
    x=self.conv1(x)
    x=self.attn(x)
    return x  

In [ ]:
class decoder(nn.Module):
    def __init__(self,input_channels,dilation):
        super(decoder,self).__init__()
        self.conv1=bn_conv(input_channels,input_channels//2,dilation)
        self.conv2=bn_conv(input_channels//2,input_channels//4,dilation)

    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        return x

In [ ]:
class final_decoder(nn.Module):
    def __init__(self,input_channels,dilation):
        super(final_decoder,self).__init__()
        self.conv1=bn_conv(input_channels,input_channels//2,dilation)
        self.conv2=nn.Conv2d(input_channels//2,64,kernel_size=3,stride=1,padding=1)
        self.conv3=nn.Conv2d(64,2,1)

    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        return torch.sigmoid(x)

In [ ]:
class D2MSA(nn.Module):
    def __init__(self,input_channels):
        super(D2MSA,self).__init__()
        self.encoder=ResNet(input_channels,BasicBlock,[1,1,1,1])
        self.bn_conv0=attn_pyram(512)
        
        self.decoder1=decoder(1024,8)
        self.decoder2=decoder(512,6)
        self.decoder3=decoder(256,4)
        self.decoder4=final_decoder(128,2)
        
        self.dec1=dec_sup(1024)
        self.dec2=dec_sup(512)
        self.dec3=dec_sup(256)
           
        self.upconv1=upconv_block(512,512)
        self.upconv2=upconv_block(256,256)
        self.upconv3=upconv_block(128,128)
        self.upconv4=upconv_block(64,64)


    def forward(self,x):
        
        x1,x2,x3,x4,x5,xe_1,xe_2,xe_3,xe_4=self.encoder(x)
        
        x5=self.bn_conv0(x5)        

        x5=self.upconv1(x5)
        x_4=self.decoder1(torch.cat((x5,x4),1))
        xd_4=self.dec1(torch.cat((x5,x4),1))

        x_4=self.upconv2(x_4)
        x_3=self.decoder2(torch.cat((x_4,x3),1))
        xd_3=self.dec2(torch.cat((x_4,x3),1))

        x_3=self.upconv3(x_3)
        x_2=self.decoder3(torch.cat((x_3,x2),1))
        xd_2=self.dec3(torch.cat((x_3,x2),1))

        x_2=self.upconv4(x_2)
        x_out=self.decoder4(torch.cat((x_2,x1),1))
        
        return x_out, (xd_2,xd_3,xd_4), (xe_1,xe_2,xe_3,xe_4) 
  

In [ ]:
model=D2MSA(3).cuda()

In [ ]:
class IoU(nn.Module):
    def __init__(self, threshold=0.5):
        super(IoU, self).__init__()
        self.threshold = threshold
 
    def forward(self, target, input):
       
        eps = 1e-10
        input_ = (input > self.threshold).data.float()
        target_ = (target > self.threshold).data.float()
 
        intersection = torch.clamp(input_ * target_, 0, 1)
        union = torch.clamp(input_ + target_, 0, 1)
 
        if torch.mean(intersection).lt(eps):
            return torch.Tensor([0., 0., 0.,0.0, 0.])
        else:
            acc = torch.mean((input_ == target_).data.float())
            iou = torch.mean(intersection) / torch.mean(union)
            recall = torch.mean(intersection) / torch.mean(target_)
            precision = torch.mean(intersection) / torch.mean(input_)
            f1=(2.*recall*precision)/(recall+precision)
            return torch.Tensor([acc, recall, precision, f1, iou])
iou=IoU()

In [ ]:
class dice_bce_loss(nn.Module):
    def __init__(self, batch=True):
        super(dice_bce_loss, self).__init__()
        self.batch = batch
        self.bce_loss = nn.BCELoss()
 
    def soft_dice_coeff(self, y_true, y_pred):
        smooth = 1.0  # may change
        if self.batch:
            i = torch.sum(y_true)
            j = torch.sum(y_pred)
            intersection = torch.sum(y_true * y_pred)
        else:
            i = y_true.sum(1).sum(1).sum(1)
            j = y_pred.sum(1).sum(1).sum(1)
            intersection = (y_true * y_pred).sum(1).sum(1).sum(1)
        score = (2. * intersection + smooth) / (i + j + smooth)
        # score = (intersection + smooth) / (i + j - intersection + smooth)#iou
        return score.mean()
 
    def soft_dice_loss(self, y_true, y_pred):
        loss = 1 - self.soft_dice_coeff(y_true, y_pred)
        return loss
 
    def __call__(self, y_pred, y_true):
        a = self.bce_loss(y_pred, y_true)
        b = self.soft_dice_loss(y_true, y_pred)
        return a + b

In [ ]:
def soft_loss_region(preds,label):
  dice_loss=dice_bce_loss()
  loss1=dice_loss(preds[0][:,0:1,:,:],label)
  loss2=dice_loss(preds[1][:,0:1,:,:],label)
  loss3=dice_loss(preds[2][:,0:1,:,:],label)
  loss4=dice_loss(preds[3][:,0:1,:,:],label)
  loss=loss1+loss2+loss3+loss4
  return loss

def soft_loss_bound(preds,label):
  dice_loss=dice_bce_loss()
  loss1=dice_loss(preds[0][:,1:2,:,:],label)
  loss2=dice_loss(preds[1][:,1:2,:,:],label)
  loss3=dice_loss(preds[2][:,1:2,:,:],label)
  loss4=dice_loss(preds[3][:,1:2,:,:],label)
  loss=loss1+loss2+loss3+loss4
  return loss


def hard_loss_region(pred1,preds,label):
  dice_loss=dice_bce_loss()
  b,c,h,w=label.shape
  loss1=dice_loss(pred1[:,0:1,:,:],label)
  label=torch.nn.functional.interpolate(label, size=(h//2,w//2), scale_factor=None, mode='nearest')
  loss2=dice_loss(preds[0][:,0:1,:,:],label)
  label=torch.nn.functional.interpolate(label, size=(h//4,w//4), scale_factor=None, mode='nearest')
  loss3=dice_loss(preds[1][:,0:1,:,:],label)
  label=torch.nn.functional.interpolate(label, size=(h//8,w//8), scale_factor=None, mode='nearest')
  loss4=dice_loss(preds[2][:,0:1,:,:],label)
  loss=loss1+loss2+loss3+loss4
  return loss  

def hard_loss_bound(pred1,preds,label):
  dice_loss=dice_bce_loss()
  b,c,h,w=label.shape
  loss1=dice_loss(pred1[:,1:2,:,:],label)
  label=torch.nn.functional.interpolate(label, size=(h//2,w//2), scale_factor=None, mode='nearest')
  loss2=dice_loss(preds[0][:,1:2,:,:],label)
  label=torch.nn.functional.interpolate(label, size=(h//4,w//4), scale_factor=None, mode='nearest')
  loss3=dice_loss(preds[1][:,1:2,:,:],label)
  label=torch.nn.functional.interpolate(label, size=(h//8,w//8), scale_factor=None, mode='nearest')
  loss4=dice_loss(preds[2][:,1:2,:,:],label)
  loss=loss1+loss2+loss3+loss4
  return loss   

In [ ]:
def loss_fn_region(pred1,preds_dec,preds_enc,label):
  loss_enc=soft_loss_region(preds_enc,label)
  loss_dec=hard_loss_region(pred1,preds_dec,label)
  loss=loss_enc+loss_dec
  return loss 

def loss_fn_bound(pred1,preds_dec,preds_enc,label):
  loss_enc=soft_loss_bound(preds_enc,label)
  loss_dec=hard_loss_bound(pred1,preds_dec,label)
  loss=loss_enc+loss_dec
  return loss   

In [ ]:
def train_one_epoch(model,train_dl,learn):
  opt = torch.optim.Adam(model.parameters(),lr=learn)  
  running_loss_image=0.0
  metric_epoch=0.0
  model.train()
  for a,b in train_dl:
    a=a.float()
    b=b.float()
    pred1,preds_dec,preds_enc=model(a.cuda())
    loss=loss_fn_region(pred1,preds_dec,preds_enc,b[:,0:1,:,:].cuda()) + loss_fn_bound(pred1,preds_dec,preds_enc,b[:,1:2,:,:].cuda())
    opt.zero_grad()
    loss.backward()
    opt.step()
    running_loss_image += loss
    metric=iou(b[:,0:1,:,:],pred1[:,0:1,:,:].detach().cpu())
    metric_epoch += metric
  running_loss_image/=len(train_dl)
  metric_epoch /= len(train_dl)  
  
  return model, metric_epoch, running_loss_image

def validate_one_epoch(model,test_dl):
    running_loss_image=0.0
    metric_epoch=0.0
    model.train()
    with torch.no_grad():
      for a,b in test_dl:
        a=a.float()
        b=b.float()
        pred1,preds_dec,preds_enc=model(a.cuda())
        loss=loss_fn_region(pred1,preds_dec,preds_enc,b[:,0:1,:,:].cuda()) + loss_fn_bound(pred1,preds_dec,preds_enc,b[:,1:2,:,:].cuda())
        running_loss_image += loss
        metric=iou(b[:,0:1,:,:],pred1[:,0:1,:,:].detach().cpu())
        metric_epoch += metric  
    running_loss_image/=len(test_dl)
    metric_epoch /= len(test_dl)

    return metric_epoch, running_loss_image        

In [ ]:
def train_epochs_net(model,train_dl,test_dl,epoches,learn,path):
    max_accuracy=0.0
    for i in range(epoches):  
        model, iou_train, loss_train=train_one_epoch(model,train_dl,learn)
        iou_test, loss_test=validate_one_epoch(model,test_dl)
        print('epoch finished' +" " + str(i+1))
        print(f'train_loss: {loss_train:.6f}, train_iou: {iou_train[4]:.6f}')
        print(f'test_loss: {loss_test:.6f}, test_iou: {iou_test[4]:.6f}')
        print()
        if iou_test[4]>max_accuracy:
            max_accuracy=iou_test[4]
            path_final=os.path.join(path,
                                    f"epoch{i}_test_iou{iou_test[4]:.4f}")
            torch.save(model.state_dict(), path_final)    
        # path_final=os.path.join(path,
        #                             f"epoch{i}_test_loss{loss_test:.4f}.pth")
        # torch.save(model.state_dict(), path_final)  

In [ ]:
def get_fast_aji(true, pred):
    """AJI version distributed by MoNuSeg, has no permutation problem but suffered from 
    over-penalisation similar to DICE2.
    Fast computation requires instance IDs are in contiguous orderding i.e [1, 2, 3, 4] 
    not [2, 3, 6, 10]. Please call `remap_label` before hand and `by_size` flag has no 
    effect on the result.
    """
    true = np.copy(true)  # ? do we need this
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_inter = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )
    pairwise_union = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background
            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            pairwise_inter[true_id - 1, pred_id - 1] = inter
            pairwise_union[true_id - 1, pred_id - 1] = total - inter

    pairwise_iou = pairwise_inter / (pairwise_union + 1.0e-6)
    # pair of pred that give highest iou for each true, dont care
    # about reusing pred instance multiple times
    paired_pred = np.argmax(pairwise_iou, axis=1)
    pairwise_iou = np.max(pairwise_iou, axis=1)
    # exlude those dont have intersection
    paired_true = np.nonzero(pairwise_iou > 0.0)[0]
    paired_pred = paired_pred[paired_true]
    # print(paired_true.shape, paired_pred.shape)
    overall_inter = (pairwise_inter[paired_true, paired_pred]).sum()
    overall_union = (pairwise_union[paired_true, paired_pred]).sum()

    paired_true = list(paired_true + 1)  # index to instance ID
    paired_pred = list(paired_pred + 1)
    # add all unpaired GT and Prediction into the union
    unpaired_true = np.array(
        [idx for idx in true_id_list[1:] if idx not in paired_true]
    )
    unpaired_pred = np.array(
        [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    )
    for true_id in unpaired_true:
        overall_union += true_masks[true_id].sum()
    for pred_id in unpaired_pred:
        overall_union += pred_masks[pred_id].sum()

    aji_score = overall_inter / overall_union
    return aji_score


In [ ]:
def get_fast_dice_2(true, pred):
    """Ensemble dice."""
    true = np.copy(true)
    pred = np.copy(pred)
    true_id = list(np.unique(true))
    pred_id = list(np.unique(pred))

    overall_total = 0
    overall_inter = 0

    true_masks = [np.zeros(true.shape)]
    for t in true_id[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [np.zeros(true.shape)]
    for p in pred_id[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    for true_idx in range(1, len(true_id)):
        t_mask = true_masks[true_idx]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        try:  # blinly remove background
            pred_true_overlap_id.remove(0)
        except ValueError:
            pass  # just mean no background
        for pred_idx in pred_true_overlap_id:
            p_mask = pred_masks[pred_idx]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            overall_total += total
            overall_inter += inter

    return 2 * overall_inter / overall_total


In [ ]:
def evaluation(model,test_dl):
  model.eval()
  aji_score=0.0
  dice_score=0.0
  with torch.no_grad():
    for a,b in test_dl:
      aji_batch=0.0
      dice_batch=0.0
      a=a.float()
      b=b.float()
      pred1,_,_=model(a.cuda())
      pred1=pred1.detach().cpu().numpy()
      b=b.detach().cpu().numpy()
      for i in range(pred1.shape[0]):
        pred=(pred1[i,0,:,:]>0.5)*1.0
        gt_label=skimage.morphology.label(b[i,0,:,:], return_num=False)
        pred_label=skimage.morphology.label(pred, return_num=False)
        aji=get_fast_aji(gt_label,pred_label)
        dice_obj=get_fast_dice_2(gt_label,pred_label)
        aji_batch += aji
        dice_batch += dice_obj
      aji_batch=aji_batch/(pred1.shape[0])
      dice_batch=dice_batch/(pred1.shape[0])
      aji_score += aji_batch
      dice_score += dice_batch
  aji_score /= len(test_dl)
  dice_score /= len(test_dl)
  return aji_score, dice_score  

In [ ]:
aj,di=evaluation(model,test_dl)

In [ ]:
def get_fast_pq(true, pred, match_iou=0.5):
    """`match_iou` is the IoU threshold level to determine the pairing between
    GT instances `p` and prediction instances `g`. `p` and `g` is a pair
    if IoU > `match_iou`. However, pair of `p` and `g` must be unique 
    (1 prediction instance to 1 GT instance mapping).
    If `match_iou` < 0.5, Munkres assignment (solving minimum weight matching
    in bipartite graphs) is caculated to find the maximal amount of unique pairing. 
    If `match_iou` >= 0.5, all IoU(p,g) > 0.5 pairing is proven to be unique and
    the number of pairs is also maximal.    
    
    Fast computation requires instance IDs are in contiguous orderding 
    i.e [1, 2, 3, 4] not [2, 3, 6, 10]. Please call `remap_label` beforehand 
    and `by_size` flag has no effect on the result.
    Returns:
        [dq, sq, pq]: measurement statistic
        [paired_true, paired_pred, unpaired_true, unpaired_pred]: 
                      pairing information to perform measurement
                    
    """
    assert match_iou >= 0.0, "Cant' be negative"

    true = np.copy(true)
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_iou = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise iou
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background
            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            iou = inter / (total - inter)
            pairwise_iou[true_id - 1, pred_id - 1] = iou
    #
    if match_iou >= 0.5:
        paired_iou = pairwise_iou[pairwise_iou > match_iou]
        pairwise_iou[pairwise_iou <= match_iou] = 0.0
        paired_true, paired_pred = np.nonzero(pairwise_iou)
        paired_iou = pairwise_iou[paired_true, paired_pred]
        paired_true += 1  # index is instance id - 1
        paired_pred += 1  # hence return back to original
    else:  # * Exhaustive maximal unique pairing
        #### Munkres pairing with scipy library
        # the algorithm return (row indices, matched column indices)
        # if there is multiple same cost in a row, index of first occurence
        # is return, thus the unique pairing is ensure
        # inverse pair to get high IoU as minimum
        paired_true, paired_pred = linear_sum_assignment(-pairwise_iou)
        ### extract the paired cost and remove invalid pair
        paired_iou = pairwise_iou[paired_true, paired_pred]

        # now select those above threshold level
        # paired with iou = 0.0 i.e no intersection => FP or FN
        paired_true = list(paired_true[paired_iou > match_iou] + 1)
        paired_pred = list(paired_pred[paired_iou > match_iou] + 1)
        paired_iou = paired_iou[paired_iou > match_iou]

    # get the actual FP and FN
    unpaired_true = [idx for idx in true_id_list[1:] if idx not in paired_true]
    unpaired_pred = [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    # print(paired_iou.shape, paired_true.shape, len(unpaired_true), len(unpaired_pred))

    #
    tp = len(paired_true)
    fp = len(unpaired_pred)
    fn = len(unpaired_true)
    # get the F1-score i.e DQ
    dq = tp / (tp + 0.5 * fp + 0.5 * fn)
    # get the SQ, no paired has 0 iou so not impact
    sq = paired_iou.sum() / (tp + 1.0e-6)

    return [dq, sq, dq * sq], [paired_true, paired_pred, unpaired_true, unpaired_pred]
